# CNMF Componenent evaluation

This is similar to the last visualization in `cnmf_viz.ipynb` but combines it with component evaluation

View cmaps with `r_values`, `cnn_preds`, `comp_SNR`

Click on components and press `"g"` to add to good, `"b"` to add to bad.

In [1]:
from mesmerize_core import *
import numpy as np
import pandas as pd

2023-09-13 17:31:12.595805: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 17:31:12.645118: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 17:31:12.645907: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 17:31:13.444315: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from fastplotlib import ImageWidget, Plot, GridPlot
import fastplotlib as fpl
from fastplotlib.utils import make_colors
from ipywidgets import VBox, IntSlider, Layout

In [3]:
pd.options.display.max_colwidth = 120

# Paths

In [4]:
# for this demo set this dir as the path to your `caiman_data` dir
set_parent_raw_data_path("/home/kushal/caiman_data/")

# batch path
batch_path = get_parent_raw_data_path().joinpath("mesmerize-batch/batch.pickle")

## Load batch

In [5]:
df = load_batch(batch_path)
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df_mean_projection.n...,2023-01-28T14:34:58,2023-01-28T14:39:47,64.96 sec,None,249713a8-757b-467e-8ae2-4108160378df
1,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 't...",{'mean-projection-path': 763c9f74-3847-4b01-af9c-550e2f46f2e5/763c9f74-3847-4b01-af9c-550e2f46f2e5_mean_projection.n...,2023-01-28T14:43:42,2023-01-28T14:44:54,53.65 sec,None,763c9f74-3847-4b01-af9c-550e2f46f2e5
2,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 'ts...",{'mean-projection-path': 793c72ed-67b7-42dd-bc6c-70edcaf81cb2/793c72ed-67b7-42dd-bc6c-70edcaf81cb2_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:46:13,73.42 sec,None,793c72ed-67b7-42dd-bc6c-70edcaf81cb2
3,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 't...",{'mean-projection-path': d752e113-6241-4866-bb96-c293bbd57e95/d752e113-6241-4866-bb96-c293bbd57e95_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:47:32,70.55 sec,None,d752e113-6241-4866-bb96-c293bbd57e95
4,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 'ts...",{'mean-projection-path': 801ee5c5-bdbb-4b7b-850c-f57d535eee02/801ee5c5-bdbb-4b7b-850c-f57d535eee02_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:48:54,76.34 sec,None,801ee5c5-bdbb-4b7b-850c-f57d535eee02
5,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 't...",{'mean-projection-path': c0f30b7e-5e6c-4177-81bc-042fc410850a/c0f30b7e-5e6c-4177-81bc-042fc410850a_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:50:22,81.35 sec,None,c0f30b7e-5e6c-4177-81bc-042fc410850a
6,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 'ts...",{'mean-projection-path': e666e0b8-323c-487d-91a8-5f798a1fd15a/e666e0b8-323c-487d-91a8-5f798a1fd15a_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:51:46,76.99 sec,None,e666e0b8-323c-487d-91a8-5f798a1fd15a
7,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 't...",{'mean-projection-path': cf2371fd-0fa1-40d9-8d60-c567000f6f4a/cf2371fd-0fa1-40d9-8d60-c567000f6f4a_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:53:05,72.49 sec,None,cf2371fd-0fa1-40d9-8d60-c567000f6f4a
8,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...

In [6]:
# You can change this to plot the outputs for different batch items
index = 1

# get the motion corrected input movie as a memmap
cnmf_movie = df.iloc[index].caiman.get_input_movie()

# we can get the contours of the spatial components
contours, coms = df.iloc[index].cnmf.get_contours("all", swap_dim=False)

# and temporal components
temporal = df.iloc[index].cnmf.get_temporal("all")

ixs_good = df.iloc[index].cnmf.get_good_components()
ixs_bad = df.iloc[index].cnmf.get_bad_components()

cnmf_obj = df.iloc[index].cnmf.get_output()

In [7]:
np.sort(cnmf_obj.estimates.r_values)

array([-0.19361374, -0.00949474,  0.00407735,  0.00752061,  0.0455527 ,
        0.06338128,  0.14284822,  0.178913  ,  0.18812583,  0.20340175,
        0.20821301,  0.24875042,  0.28352687,  0.3000297 ,  0.31562525,
        0.32752661,  0.34441137,  0.36287603,  0.37505743,  0.37657836,
        0.37759387,  0.38364619,  0.38382803,  0.39640055,  0.400456  ,
        0.41895753,  0.44048346,  0.4434426 ,  0.44839013,  0.44913824,
        0.45965112,  0.47577452,  0.47721189,  0.48324424,  0.48630713,
        0.4882223 ,  0.50031692,  0.50290923,  0.50680938,  0.50682529,
        0.50868902,  0.51604752,  0.51657422,  0.51796221,  0.51894629,
        0.52217315,  0.53642356,  0.54972932,  0.54997323,  0.55028125,
        0.55152869,  0.55249284,  0.55395974,  0.55498703,  0.55965712,
        0.56667751,  0.56816878,  0.58273456,  0.5927396 ,  0.59968885,
        0.60124461,  0.60183404,  0.60691893,  0.62157624,  0.6240117 ,
        0.62529828,  0.63185491,  0.63543728,  0.63690499,  0.64

In [8]:
rcm = df.iloc[index].cnmf.get_rcm()
corr_img = df.iloc[index].caiman.get_corr_image()

In [9]:
class DummyMovie:
    def __init__(self, image: np.ndarray, shape, ndim, size):
        self.image = image
        self.shape = shape
        self.ndim = ndim
        self.size = size
        
    def __getitem__(self, index: int):
        return self.image

In [26]:
from ipywidgets import RadioButtons, FloatSlider

In [35]:
# 2 x 2 gridplot
# |=====================|
# |  movie |    rcm     |
# |---------------------|
# | corr  |  temporal   |
# |=====================|
cnmf_grid = GridPlot(
    shape=(1, 3), 
    controllers="sync", 
    names=[["movie", "rcm", "corr"]]
)

corr_img_movie = DummyMovie(corr_img, shape=cnmf_movie.shape, ndim=cnmf_movie.ndim, size=cnmf_movie.size)

iw = fpl.ImageWidget(
    [cnmf_movie, rcm, corr_img_movie],
    vmin_vmax_sliders=True,
    names=["movie", "rcm", "corr"],
    cmap="gnuplot2"
)

snr_comps_log = np.log10(cnmf_obj.estimates.SNR_comp)
r_values = cnmf_obj.estimates.r_values
cnn_preds = cnmf_obj.estimates.cnn_preds

# add contours to both movie and rcm subplots
contours_movie = iw.gridplot["movie"].add_line_collection(
    contours,
    cmap="spring",
    cmap_values=np.log(snr_comps),
    thickness=3,
    name="contours",
)

contours_rcm = iw.gridplot["rcm"].add_line_collection(
    contours,
    cmap="spring",
    cmap_values=np.log(snr_comps),
    thickness=3,
    name="contours",
)

contours_corr = iw.gridplot["corr"].add_line_collection(
    contours,
    cmap="spring",
    cmap_values=np.log(snr_comps),
    thickness=3,
    name="contours",
)

# plot single temporal, just like before
plot_temporal_single = Plot()
temporal_graphic = plot_temporal_single.add_line_collection(temporal, colors="w")

radio_eval_metrics = RadioButtons(
    options=["snr", "r_values", "cnn_preds"],
    value="snr",
    description="current eval colors"
)

def change_eval_metric(change):
    value = change["new"]
    
    current_bad_ixs = cnmf_obj.estimates.idx_components_bad
    
    if value == "snr":
        contours_rcm.cmap_values = snr_comps_log
        contours_movie.cmap_values = snr_comps_log
        contours_corr.cmap_values = snr_comps_log
        
    elif value == "r_values":
        contours_rcm.cmap_values = r_values
        contours_movie.cmap_values = r_values
        contours_corr.cmap_values = r_values
    
    elif value == "cnn_preds":
        contours_rcm.cmap_values = cnn_preds
        contours_movie.cmap_values = cnn_preds
        contours_corr.cmap_values = cnn_preds
        
    contours_rcm[current_bad_ixs].colors = "w"
    contours_movie[current_bad_ixs].colors = "w"
    contours_corr[current_bad_ixs].colors = "w"
        
radio_eval_metrics.observe(change_eval_metric, "value")

slider_min_snr = FloatSlider(
    min=snr_comps.min(), 
    max=snr_comps.max(), 
    value=cnmf_obj.params.get_group("quality")["min_SNR"],
    step=(snr_comps.max() - snr_comps.min()) / 100,
    description="min_SNR",
)


def update_eval(change):
    new_params = {
        "min_SNR": slider_min_snr.value
    }
    
    cnmf_obj.params.quality.update(new_params)
    cnmf_obj.estimates.filter_components(cnmf_movie, cnmf_obj.params)
    change_eval_metric({"new": radio_eval_metrics.value})
    

slider_min_snr.observe(update_eval, "value")
    
VBox([plot_temporal_single.show(), iw.show(), radio_eval_metrics, slider_min_snr])

RFBOutputContext()

RFBOutputContext()

RFBOutputContext()

In [48]:
df.iloc[1].cnmf.run_detrend_dfof()

In [49]:
df.iloc[1].cnmf.get_output_path()

PosixPath('/home/kushal/caiman_data/mesmerize-batch/763c9f74-3847-4b01-af9c-550e2f46f2e5/763c9f74-3847-4b01-af9c-550e2f46f2e5.hdf5')

In [50]:
import h5py

In [51]:
h5_path = df.iloc[1].cnmf.get_output_path()

f = h5py.File(h5_path)

In [52]:
f.keys()

<KeysViewHDF5 ['dims', 'dview', 'estimates', 'params', 'remove_very_bad_comps', 'skip_refinement']>

In [53]:
f["estimates"].keys()

<KeysViewHDF5 ['A', 'A_thr', 'Ab', 'Ab_dense', 'AtA', 'AtY_buf', 'C', 'CC', 'CY', 'C_on', 'Cf', 'F_dff', 'OASISinstances', 'R', 'S', 'SNR_comp', 'W', 'YrA', 'Yr_buf', 'b', 'b0', 'bl', 'c1', 'center', 'cnn_preds', 'dims', 'discarded_components', 'ecc', 'f', 'g', 'idx_components', 'idx_components_bad', 'ind_new', 'lam', 'mn', 'neurons_sn', 'noisyC', 'nr', 'psx', 'r_values', 'rho_buf', 'shifts', 'sn', 'sv', 'vr']>

In [55]:
f["estimates"]["F_dff"][:]

array([[-0.02004358,  0.00981987, -0.0128868 , ..., -0.06741492,
        -0.02886321,  0.01014399],
       [-0.1220207 , -0.01579472,  0.0833003 , ..., -0.00495933,
         0.10249602,  0.01094983],
       [-0.03787187, -0.03436601,  0.04886   , ...,  0.04343703,
         0.1316829 ,  0.09539957],
       ...,
       [-0.04911215,  0.11291489,  0.07802214, ...,  0.12369123,
         0.06761335, -0.09787255],
       [-0.0183316 , -0.18923914, -0.14147669, ..., -0.06016275,
        -0.00673185, -0.07319726],
       [-0.07732914,  0.00380741, -0.07273969, ...,  0.20057677,
        -0.05499444,  0.03450985]])

In [58]:
f["estimates"]["F_dff"].shape

(155, 3000)